# Calculation of CFIM, QFIM and HCRB

This is the source codes for the example discussed in Sec. III A and B in Ref. [1].

The Hamiltonian of a two-qubit system with $XX$ coupling is 
\begin{align}
H=\omega_1\sigma_3^{(1)}+\omega_2\sigma_3^{(2)}+g\sigma_1^{(1)}\sigma_1^{(2)},
\end{align}

where $\omega_1$, $\omega_2$ are the frequencies of the first and second qubit, $\sigma_i^{(1)}=
\sigma_i\otimes I$ and $\sigma_i^{(2)}=I\otimes\sigma_i$ for $i=1,2,3$. $\sigma_1$, $\sigma_2$, $\sigma_3$ 
are Pauli matrices and $I$ denotes the identity matrix. The dynamics is described by the master equation 
\begin{align}
\partial_t\rho=-i[H, \rho]+\sum_{i=1,2}\gamma_i\left(\sigma_3^{(i)}\rho\sigma_3^{(i)}-\rho\right)
\end{align}

with $\gamma_i$ the decay rate for the $i$th qubit.

The probe state is taken as $\frac{1}{\sqrt{2}}(|00\rangle+|11\rangle)$ and the weight matrix is set to be
identity. The measurement for $\mathrm{Tr}(W\mathcal{I^{-1}})$ is $\{\Pi_1$, $\Pi_2$, $I-\Pi_1-\Pi_2\}$ 
with $\Pi_1=0.85|00\rangle\langle 00|$ and $\Pi_2=0.1|\!+\!+\rangle\langle+\!+\!|$. Here 
$|\pm\rangle:=\frac{1}{\sqrt{2}}(|0\rangle\pm|1\rangle)$ with $|0\rangle$ $(|1\rangle)$ the eigenstate of 
$\sigma_3$ with respect to the eigenvalue $1$ ($-1$).

[1] M. Zhang et al., QuanEstimation: an open-source toolkit for quantum parameter estimation,
arXiv:2022.xxxx.

In [ ]:
from quanestimation import *
import numpy as np

# initial state
psi0 = np.array([1., 0., 0., 1.])/np.sqrt(2)
rho0 = np.dot(psi0.reshape(-1,1), psi0.reshape(1,-1).conj())
# free Hamiltonian
omega1, omega2, g = 1.0, 1.0, 0.1
sx = np.array([[0., 1.], [1., 0.]])
sy = np.array([[0., -1.j], [1.j, 0.]]) 
sz = np.array([[1., 0.], [0., -1.]])
ide = np.array([[1., 0.], [0., 1.]])   
H0 = omega1*np.kron(sz, ide)+omega2*np.kron(ide, sz)+g*np.kron(sx, sx)
# derivatives of the free Hamiltonian on omega2 and g
dH = [np.kron(ide, sz), np.kron(sx, sx)] 
# dissipation
decay = [[np.kron(sz,ide), 0.05], [np.kron(ide,sz), 0.05]]
# measurement
m1 = np.array([1., 0., 0., 0.])
M1 = 0.85*np.dot(m1.reshape(-1,1), m1.reshape(1,-1).conj())
M2 = 0.1*np.ones((4, 4))
M = [M1, M2, np.identity(4)-M1-M2]
# time length for the evolution
tspan = np.linspace(0., 10., 1000)
# dynamics
dynamics = Lindblad(tspan, rho0, H0, dH, decay)
rho, drho = dynamics.expm()
# weight matrix
W = np.identity(2)
# calculation of the CFIM, QFIM and HCRB
F, I, f_HCRB, f_NHB = [], [], [], []
for ti in range(1, 1000):
    # CFIM
    I_tp = CFIM(rho[ti], drho[ti], M=M)
    I.append(I_tp)
    # QFIM
    F_tp = QFIM(rho[ti], drho[ti])
    F.append(F_tp)
    # HCRB
    f_tp1 = HCRB(rho[ti], drho[ti], W, eps=1e-6)
    f_HCRB.append(f_tp1)
    # NHB
    f_tp2 = NHB(rho[ti], drho[ti], W)
    f_NHB.append(f_tp2)